In [1]:
# Linear Regression

In [2]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [3]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [4]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
def get_expiry(f_today):
    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [355]:
bnf_pandas = pd.read_csv("../data/midcp.csv")
# bnf_pandas = pd.read_csv("../data/finnifty_1hr_tv (2).csv")
# bnf_pandas = pd.read_csv('../data/midcp_1hr_tv (4).csv')
# bnf_pandas = pd.read_csv('../data/finnifty_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/bnf_fut_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')

In [356]:
# bnf.tail()

In [357]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]
# bnf_pandas.drop(columns=["datetime"], inplace=True)
# bnf_pandas

In [358]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [359]:
# # For crude oil Data
# bnf1 = pd.DataFrame(bnf_pandas)
# bnf1 = bnf1.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
# bnf1['datetime'] = pd.to_datetime(bnf1['datetime'])
# bnf1['index'] = bnf1['datetime']
# bnf1.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close'}, inplace=True)
# bnf = pl.DataFrame(bnf1)
# print(type(bnf))
# bnf

In [360]:
# bnf['datetime'] = pd.to_datetime(bnf['datetime'])
# bnf = bnf.drop(columns=['Unnamed: 0'])
# bnf.set_index(bnf['datetime'], inplace=True)
# bnf
# bnf = bnf.with_columns(pl.col('datetime').str.to_datetime(format='%Y-%m-%dT%H:%M:%S.%f'))
# print(bnf)
# bnf = bnf.with_columns(pl.col('datetime').cast(pl.DateTime))

# Set 'datetime' column as index
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")

# Now 'datetime' is set as the index
# bnf

In [361]:
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
# bnf

In [362]:
bnf.tail()

open,high,low,close,volume,index,datetime
f64,f64,f64,f64,i64,datetime[ns],datetime[ns]
11762.3,11763.25,11756.65,11758.2,0,2024-06-07 15:25:00,2024-06-07 15:25:00
11758.3,11758.9,11753.7,11755.85,0,2024-06-07 15:26:00,2024-06-07 15:26:00
11753.9,11757.2,11752.05,11756.4,0,2024-06-07 15:27:00,2024-06-07 15:27:00
11756.45,11757.4,11748.75,11753.7,0,2024-06-07 15:28:00,2024-06-07 15:28:00
11753.2,11753.85,11747.05,11748.6,0,2024-06-07 15:29:00,2024-06-07 15:29:00


In [363]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )

# ohlc_resampled = resample(bnf, '60m', pd.Timedelta(minutes=15))

# bnf_df = bnf.to_pandas()
# bnf_df.set_index(bnf_df['datetime'], inplace=True)
# ohlc_15min = bnf_df.resample('60min').agg({
#     'open': 'first',
#     'high': 'max',
#     'low': 'min',
#     'close': 'last',
# })

# ohlc_15min.dropna(inplace=True)

# ohlc_15min.reset_index(inplace=True)

# bnf_1hr = ohlc_15min
# bnf_1hr = ohlc_resampled
# bnf_final = bnf_1hr.to_pandas()
# bnf_final['datetime'] = pd.to_datetime(bnf_final['datetime'])
# bnf_final
# bnf_1hr

In [364]:
# bnf_final = bnf
# bnf_final

In [365]:
# # bnf_final = bnf_1hr
# bnf_final['datetime'] = pd.to_datetime(bnf_final['datetime'])
# bnf_final

In [366]:
import pandas as pd
import numpy as np

def linear_regression_vectorized(df, period, trailing):
    
    if 'close' not in df.columns:
        print(df.columns)
        raise ValueError("The DataFrame must contain a 'close' column.")

    df['close'] = df['close'].astype(float)
    x = np.arange(period)
    x_sum = np.sum(x)
    x_squared_sum = np.sum(x**2)
    
    denominator = period * x_squared_sum - x_sum**2
    
    y_rolling = df['close'].rolling(window=period)
    
    y_sum = y_rolling.sum()
    xy_sum = y_rolling.apply(lambda y: np.dot(x, y), raw=True)
    
    slope = (period * xy_sum - x_sum * y_sum) / denominator
    intercept = (y_sum - slope * x_sum) / period
    
    predicted_values = intercept + slope * (period - 1)
    df['LR'] = predicted_values

    x = np.arange(trailing)
    x_sum = np.sum(x)
    x_squared_sum = np.sum(x**2)
    
    denominator = trailing * x_squared_sum - x_sum**2
    
    y_rolling = df['close'].rolling(window=trailing)
    
    y_sum = y_rolling.sum()
    xy_sum = y_rolling.apply(lambda y: np.dot(x, y), raw=True)
    
    slope = (trailing * xy_sum - x_sum * y_sum) / denominator
    intercept = (y_sum - slope * x_sum) / trailing
    
    predicted_values = intercept + slope * (trailing - 1)
    df['LR2'] = predicted_values
    
    return df

# data = linear_regression_vectorized(bnf.to_pandas(), 20)

In [367]:
# # Example usage
# df = pd.DataFrame({
#     'Date': pd.date_range(start='2023-01-01', periods=50, freq='D'),
#     'Close': np.random.uniform(100, 200, 50)
# }).

# df.set_index('Date', inplace=True)

# # Apply the optimized linear regression function
# regression_values = linear_regression_vectorized(df, period=14)
# df['Linear_Regression_Close'] = regression_values

# # Display the DataFrame with the new column
# print(df.tail(10))

In [368]:
def generate_signals1(df, candles_in_num):
    df["close"] = pd.to_numeric(df["close"], errors="coerce")

    df["Buy_Signal"] = 0
    buy_signal_mask = df["LR"] > df["LR"].shift(1)

    for i in range(1, candles_in_num):
        buy_signal_mask &= df["LR"].shift(i) > df["LR"].shift(i + 1)

    df.loc[buy_signal_mask, "Buy_Signal"] = 1

    return df

In [369]:
def generate_signals2(df, candles_in_num):
    df["c"] = pd.to_numeric(df["c"], errors="coerce")
    
    df["Sell_Signal"] = 0

    # Generate signals using boolean masking
    sell_signal_mask = df["LR"] < df["LR"].shift(1)

    for i in range(1, candles_in_num):
        sell_signal_mask &= df["LR"].shift(i) < df["LR"].shift(i + 1)

    df.loc[sell_signal_mask, "Sell_Signal"] = 1

    return df

In [381]:
# Positional

def execute(df, n):

    trade_book = []
    in_trade = False
    signal_entry_price = 100000
    signal_initial_sl = 0
    already_signal_exists = False
    is_trailing_active = False
    remark = ""
    portfolio_value = 1_00_00_000
    index_lev = 3

    for i in range(1, len(df)):
        points = 0
        current_candle_open = df.iloc[i]["o"]
        current_candle_high = df.iloc[i]["h"]
        current_candle_low = df.iloc[i]["l"]
        current_candle_close = df.iloc[i]["c"]
        current_linear_regression = df.iloc[i]['LR2']

        if not in_trade:
            if df.iloc[i]["Buy_Signal"] == 1:
                if not already_signal_exists:
                    # Fresh Buy Signal
                    # print(df.iloc[i])
                    # print('Fresh Buy Signal')
                    signal_entry_price = current_candle_high
                    # signal_initial_sl = current_candle_low
                    signal_initial_sl = df.iloc[i-n-1:i+1]['l'].min()
                    signal_creation_time = df.iloc[i]["datetime"]
                    already_signal_exists = True

                else:
                    if current_candle_open > signal_entry_price:
                        # Gap Up Open, SKIP trade
                        # print(df.iloc[i])
                        # print('Gap Up Open, Skip Trade')
                        already_signal_exists = False
                        signal_entry_price = 100000
                        signal_initial_sl = 0
                    elif current_candle_high < signal_entry_price:
                        # Better Candle
                        # print(df.iloc[i])
                        # print('Better Candle')
                        signal_entry_price = current_candle_high
                        # signal_initial_sl = current_candle_low
                        signal_initial_sl = df.iloc[i-n-1:i+1]['l'].min()
                        signal_creation_time = df.iloc[i]["datetime"]

                        # if (signal_entry_price - signal_initial_sl > 400):
                        #     # Skip Signal Candle Due To Big Size
                        #     already_signal_exists = False
                        #     signal_entry_price = 100000
                        #     signal_initial_sl = 0

                    elif current_candle_high > signal_entry_price:
                        # Entry Triggered
                        # print(df.iloc[i])
                        # print('Entry Triggered')
                        in_trade = True
                        entry_time = df.iloc[i]["datetime"]
                        entry_price = signal_entry_price
                        points = 0
            else:
                if df.iloc[i - 1]["Buy_Signal"] == 1:
                    # Considering the forward bias condition as well now
                    signal_entry_price = df.iloc[i - 1]["h"]
                    # signal_initial_sl = df.iloc[i - 1]["l"]
                    signal_initial_sl = df.iloc[i-n-2:i]['l'].min()
                    signal_creation_time = df.iloc[i - 1]["datetime"]
                    already_signal_exists = True

                    if current_candle_high > signal_entry_price:
                        in_trade = True
                        entry_time = df.iloc[i]["datetime"]
                        entry_price = signal_entry_price
                        points = 0
                    else:
                        # Discard Existing Signal
                        already_signal_exists = False
                        signal_entry_price = 100000
                        signal_initial_sl = 0

        if in_trade:
            trade_entry_price = signal_entry_price
            trade_initial_sl = signal_initial_sl
            trade_final_sl = signal_initial_sl

            if (
                not is_trailing_active
                and current_candle_low > current_linear_regression
            ):
                is_trailing_active = True

            if not is_trailing_active:
                if current_candle_open < trade_initial_sl:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        if current_candle_close <= trade_initial_sl:
                            in_trade = False
                            points = trade_initial_sl - trade_entry_price
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL
                        # print(df.iloc[i])
                        # print('Gap Open Outside ISL')
                        in_trade = False
                        points = current_candle_open - trade_entry_price
                        exit_price = current_candle_open
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif current_candle_low <= trade_initial_sl:
                    # Initial SL Hit
                    # print(df.iloc[i])
                    # print('Initial SL Hit')
                    in_trade = False
                    points = trade_initial_sl - trade_entry_price
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL Hit"

            else:
                trade_final_sl = max(trade_initial_sl, current_linear_regression)

                if current_candle_open <= trade_initial_sl:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        if current_candle_close <= trade_initial_sl:
                            in_trade = False
                            points = trade_initial_sl - trade_entry_price
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL
                        # print(df.iloc[i])
                        # print('Gap Open Outside ISL')
                        in_trade = False
                        points = current_candle_open - trade_entry_price
                        exit_price = current_candle_open
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif current_candle_low <= trade_initial_sl:
                    # if trade_initial_sl >= current_linear_regression:
                    # Despite Trailing, Initial SL hit
                    # print(df.iloc[i])
                    # print('Initial SL Hit')
                    in_trade = False
                    points = trade_initial_sl - trade_entry_price
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL hit"

                elif (current_candle_close <= trade_final_sl):
                    # Price Closed below TSL i.e. MA10 , TSL Hit
                    # print(df.iloc[i])
                    # print('Initial SL Hit')
                    in_trade = False
                    points = current_candle_close - trade_entry_price
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    is_trailing_active = False
                    remark = "TSL Hit"

            if points:
                
                max_qty = int(round(portfolio_value * index_lev / entry_price / 25)) * 25
                rpt_qty = portfolio_value * 0.01 / abs(entry_price - trade_initial_sl)
                # qty = min(max_qty, rpt_qty)
                qty = max_qty
                slippage = 0.0001 * (entry_price + exit_price)
                # slippage = 10
                final_points = points - slippage
                # final_points = points

                # if vol_entry * strike < 250000000:
                #     vol_remark = 'ILLIQUID'
                # elif vol_entry * strike >= 250000000:
                #     vol_remark = 'LIQUID'z
                
                trade = {
                    "Signal Generated At": signal_creation_time,
                    "Trade Type": "LONG",
                    "Entry Date": entry_time.date(),
                    "Entry Time": entry_time.time(),
                    "Entry Price": entry_price,
                    "Initial SL": trade_initial_sl,
                    "Final SL": trade_final_sl,
                    # "Target": trade_target,
                    "Exit Time": exit_time,
                    "Exit Price": exit_price,
                    "Points Captured": points,
                    "Slippages": slippage,
                    "After Costs": final_points,
                    "PnL": final_points * qty,
                    "Remarks": remark,
                    # 'Volume at Entry Candle': vol_entry,
                    # 'Volume Remark': vol_remark,
                    "Max Qty": max_qty,
                    "RPT Qty": rpt_qty,
                    'Qty': qty,
                    "Leverage": index_lev,
                    "ROI%": (final_points * qty / portfolio_value) * 100,
                    # "Max Price": max_price,
                    # "Min Price": min_price,
                    "Trade Year": entry_time.year,
                    "Trade Month": entry_time.month,
                    # "Variation": f"{signal_ma}, {no_of_candles}, {tf}, {sl}% SL",
                }
                trade_book.append(trade)
                points = 0
                in_trade = False
                already_signal_exists = False
                remark = ""
                is_trailing_active = False

    trade_book_df = pd.DataFrame(trade_book)
    return trade_book_df

In [382]:
bnf1 = bnf
bnf2 = bnf

In [383]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [384]:
bnf1 = bnf1.to_pandas()

In [385]:
# # bnf2 = bnf2.to_pandas()
# data = linear_regression_vectorized(bnf1, 80)
# signals_df1 = generate_signals1(data, 1)
# # signals_df2 = generate_signals2(bnf2)

In [386]:
# data[data['Buy_Signal'] > 0].tail()

In [387]:
# bnf1.tail()

In [388]:
bnf2 = resample(pl.DataFrame(bnf1), "15m", "0m")
data = linear_regression_vectorized(bnf2.to_pandas(), 45, 25)
# print(data.tail(10).to_string())
signals_df1 = generate_signals1(data, 3)
signals_df1 = signals_df1.rename(columns={"open": "o", "high": "h", "low": "l", "close": "c"})
new_tb = execute(signals_df1, 1)

# new_tb = execute(signals_df1)
new_tb["DD%"] = new_tb["ROI%"].cumsum() - new_tb["ROI%"].cumsum().cummax()
tradebook_buy_side = new_tb
# new_tb.tail(25)

In [389]:
# tradebook_buy_side['DD%'] = (tradebook_buy_side['ROI%'].cumsum() - tradebook_buy_side['ROI%'].cumsum().cummax())
tradebook_buy_side[tradebook_buy_side['Trade Year'] >= 2024]

,Signal Generated At,Trade Type,Entry Date,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,Slippages,After Costs,PnL,Remarks,Max Qty,RPT Qty,Qty,Leverage,ROI%,Trade Year,Trade Month,DD%
606,2024-01-01 09:30:00,LONG,2024-01-01,09:45:00,10413.6000,10390.7000,10428.0418,2024-01-01 11:30:00,10423.6500,10.0500,2.0837,7.9663,22903.0406,TSL Hit,2875,4366.8122,2875,3,0.2290,2024,1,-3.6652
607,2024-01-01 11:45:00,LONG,2024-01-01,12:00:00,10434.1000,10418.9000,10418.9000,2024-01-01 15:00:00,10418.9000,-15.2000,2.0853,-17.2853,-49695.2375,Initial SL Hit,2875,6578.9474,2875,3,-0.4970,2024,1,-4.1622
608,2024-01-03 12:30:00,LONG,2024-01-03,12:45:00,10384.6500,10358.4000,10401.7383,2024-01-03 14:15:00,10399.7500,15.1000,2.0784,13.0216,37762.5240,TSL Hit,2900,3809.5238,2900,3,0.3776,2024,1,-3.7845
609,2024-01-04 09:30:00,LONG,2024-01-04,09:45:00,10458.5000,10365.1500,10521.4651,2024-01-04 12:00:00,10517.7500,59.2500,2.0976,57.1524,164313.0781,TSL Hit,2875,1071.2373,2875,3,1.6431,2024,1,-2.1414
610,2024-01-04 12:30:00,LONG,2024-01-04,12:45:00,10518.9000,10501.2500,10581.5814,2024-01-05 10:30:00,10560.8000,41.9000,2.1080,39.7920,113407.2855,TSL Hit,2850,5665.7224,2850,3,1.1341,2024,1,-1.0073
611,2024-01-05 10:45:00,LONG,2024-01-05,11:00:00,10561.8000,10527.0500,10527.0500,2024-01-05 14:00:00,10527.0500,-34.7500,2.1089,-36.8589,-105047.8222,Initial SL Hit,2850,2877.6978,2850,3,-1.0505,2024,1,-2.0578
612,2024-01-09 10:30:00,LONG,2024-01-09,10:45:00,10501.6500,10483.1500,10483.1500,2024-01-09 10:45:00,10483.1500,-18.5000,2.0985,-20.5985,-58705.6680,Initial SL Hit,2850,5405.4054,2850,3,-0.5871,2024,1,-2.6449
613,2024-01-09 11:15:00,LONG,2024-01-09,11:30:00,10511.3500,10474.7000,10474.7000,2024-01-09 14:00:00,10474.7000,-36.6500,2.0986,-38.7486,-110433.5242,Initial SL Hit,2850,2728.5130,2850,3,-1.1043,2024,1,-3.7492
614,2024-01-10 14:45:00,LONG,2024-01-10,15:00:00,10419.6000,10354.7000,10471.4589,2024-01-11 10:45:00,10467.2000,47.6000,2.0887,45.5113,130845.0450,TSL Hit,2875,1540.8320,2875,3,1.3085,2024,1,-2.4408
615,2024-01-11 11:00:00,LONG,2024-01-11,11:15:00,10474.1000,10454.7000,10454.7000,2024-01-11 13:30:00,10454.7000,-19.4000,2.0929,-21.4929,-61792.0300,Initial SL Hit,2875,5154.6392,2875,3,-0.6179,2024,1,-3.0587


In [390]:
stats2 = generate_stats(new_tb, "...")
roi_overall, stats_overall = next(iter(stats2.items()))
stats_overall

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2017,0.0000,0,NaN,NaN,NaN,NaN,NaN,...
2018,0.0000,0,NaN,NaN,NaN,NaN,NaN,...
2019,0.0000,0,NaN,NaN,NaN,NaN,NaN,...
2020,0.0000,0,NaN,NaN,NaN,NaN,NaN,...
2021,0.0000,0,NaN,NaN,NaN,NaN,NaN,...
2022,79.6294,275,33.0909,2.4336,-0.7708,-10.4250,7.6383,...
2023,79.2719,331,38.9728,1.4591,-0.5393,-8.9522,8.8550,...
2024,38.6652,135,42.2222,1.7565,-0.7879,-8.7083,4.4400,...
Overall,197.5665,741,37.3819,1.8404,-0.6729,-10.4250,18.9513,...


In [391]:
# tradebook_buy_side.to_csv('FNF_LR.csv')

# Combining Tradebooks Below ...

In [5]:
bnf = pd.read_csv('BNF_LR.csv')
nf = pd.read_csv('NF_LR.csv')
fnf = pd.read_csv('FNF_LR.csv')
midcp = pd.read_csv('MIDCP_LR.csv')

In [6]:
bnf['Index'] = 'BANKNIFTY'
nf['Index'] = 'NIFTY'
fnf['Index'] = 'FINNIFTY'
midcp['Index'] = 'MIDCPNIFTY'

combined_df = pd.concat([bnf, nf, fnf, midcp], ignore_index=True)
combined_df.drop(columns=['Unnamed: 0'], inplace=True)
combined_df['Entry Time'] = pd.to_datetime(combined_df['Entry Time'])
# combined_df['Expiry'] = pd.to_datetime(combined_df['Expiry'])
combined_df = combined_df.sort_values(by='Entry Date')

/tmp/ipykernel_42279/1526769915.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_df['Entry Time'] = pd.to_datetime(combined_df['Entry Time'])


In [7]:
# combined_df.tail(20)

In [8]:
combined_df.loc[(combined_df['Trade Year'] < 2022), 'Qty'] /= 3
combined_df.loc[(combined_df['Trade Year'] < 2022), 'Max Qty'] /= 3
combined_df.loc[(combined_df['Trade Year'] < 2022), 'RPT Qty'] /= 3
combined_df.loc[(combined_df['Trade Year'] < 2022), 'PnL'] /= 3
combined_df.loc[(combined_df['Trade Year'] < 2022), 'ROI%'] /= 3

combined_df.loc[(combined_df['Trade Year'] >= 2022), 'Qty'] /= 4
combined_df.loc[(combined_df['Trade Year'] >= 2022), 'Max Qty'] /= 4
combined_df.loc[(combined_df['Trade Year'] >= 2022), 'RPT Qty'] /= 4
combined_df.loc[(combined_df['Trade Year'] >= 2022), 'PnL'] /= 4
combined_df.loc[(combined_df['Trade Year'] >= 2022), 'ROI%'] /= 4

/tmp/ipykernel_42279/1919142865.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 427.5   285.    135.   ... 1308.75  101.25 1278.75]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined_df.loc[(combined_df['Trade Year'] >= 2022), 'Qty'] /= 4
/tmp/ipykernel_42279/1919142865.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 427.5   285.    135.   ... 1308.75  101.25 1278.75]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined_df.loc[(combined_df['Trade Year'] >= 2022), 'Max Qty'] /= 4


In [9]:
combined_df["Cumulative PnL"] = combined_df["PnL"].cumsum()
combined_df["Cumulative ROI%"] = combined_df["ROI%"].cumsum()
combined_df["Running Max ROI%"] = combined_df["Cumulative ROI%"].cummax()
combined_df["Drawdown"] = combined_df["Cumulative ROI%"] - combined_df["Running Max ROI%"]

In [10]:
combined_df.tail(25)

,Signal Generated At,Trade Type,Entry Date,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,Slippages,After Costs,PnL,Remarks,Max Qty,RPT Qty,Qty,Leverage,ROI%,Trade Year,Trade Month,DD%,Index,Cumulative PnL,Cumulative ROI%,Running Max ROI%,Drawdown
2649,2024-05-22 15:15:00,LONG,2024-05-23,2024-09-29 09:15:00,21393.6000,21198.8000,21957.8727,2024-05-28 12:15:00,21947.0000,553.4000,4.3341,549.0659,127657.8311,TSL Hit,232.5000,128.3368,232.5000,2,1.2766,2024,5,-1.9242,FINNIFTY,40663224.4874,406.6322,406.6322,0.0000
3383,2024-05-23 11:15:00,LONG,2024-05-23,2024-09-29 11:30:00,11387.5500,11318.6000,11414.0049,2024-05-23 13:00:00,11394.0000,6.4500,2.2782,4.1718,5491.1910,TSL Hit,1316.2500,362.5816,1316.2500,6,0.0549,2024,5,-1.0538,MIDCPNIFTY,40668715.6784,406.6872,406.6872,0.0000
3385,2024-05-24 10:15:00,LONG,2024-05-24,2024-09-29 10:30:00,11428.1500,11386.1500,11503.4408,2024-05-24 13:45:00,11502.0500,73.9000,2.2930,71.6070,93984.1612,TSL Hit,1312.5000,595.2381,1312.5000,6,0.9398,2024,5,0.0000,MIDCPNIFTY,40762699.8397,407.6270,407.6270,0.0000
492,2024-05-23 15:15:00,LONG,2024-05-24,2024-09-29 09:15:00,48826.1500,48179.4000,49097.8045,2024-05-28 15:15:00,49060.1000,233.9500,9.7886,224.1614,22696.3392,TSL Hit,101.2500,38.6548,101.2500,2,0.2270,2024,5,0.0000,BANKNIFTY,40785396.1789,407.8540,407.8540,0.0000
3387,2024-05-27 12:45:00,LONG,2024-05-27,2024-09-29 13:00:00,11675.0500,11646.8500,11646.8500,2024-05-28 10:00:00,11646.8500,-28.2000,2.3322,-30.5322,-39272.0294,Initial SL Hit,1286.2500,886.5248,1286.2500,6,-0.3927,2024,5,-1.5709,MIDCPNIFTY,40746124.1495,407.4612,407.8540,-0.3927
3386,2024-05-27 09:45:00,LONG,2024-05-27,2024-09-29 10:00:00,11599.9500,11459.2000,11681.4723,2024-05-27 12:15:00,11674.0000,74.0500,2.3274,71.7226,92791.1202,TSL Hit,1293.7500,177.6199,1293.7500,6,0.9279,2024,5,0.0000,MIDCPNIFTY,40838915.2697,408.3892,408.3892,0.0000
3388,2024-05-29 13:00:00,LONG,2024-05-29,2024-09-29 13:15:00,11572.5500,11545.5000,11554.2692,2024-05-29 14:15:00,11553.5500,-19.0000,2.3126,-21.3126,-27653.1115,TSL Hit,1297.5000,924.2144,1297.5000,6,-0.2765,2024,5,-2.6770,MIDCPNIFTY,40811262.1583,408.1126,408.3892,-0.2765
2650,2024-05-30 12:45:00,LONG,2024-05-30,2024-09-29 13:15:00,21685.8500,21560.8000,21560.8000,2024-05-30 14:45:00,21560.8000,-125.0500,4.3247,-129.3747,-29594.4546,Initial SL Hit,228.7500,199.9200,228.7500,2,-0.2959,2024,5,-3.1080,FINNIFTY,40781667.7036,407.8167,408.3892,-0.5725
1519,2024-05-31 10:45:00,LONG,2024-05-31,2024-09-29 11:15:00,22554.0000,22495.8500,22550.9700,2024-05-31 14:15:00,22495.8500,-58.1500,4.5050,-62.6550,-20911.1012,Initial SL hit,333.7500,429.9226,333.7500,3,-0.2091,2024,5,-1.1997,NIFTY,40760756.6024,407.6076,408.3892,-0.7816
2651,2024-05-31 13:15:00,LONG,2024-05-31,2024-09-29 13:45:00,21725.5500,21565.7500,21721.5818,2024-05-31 14:15:00,21690.0000,-35.5500,4.3416,-39.8916,-9125.1932,TSL Hit,228.7500,156.4456,228.7500,2,-0.0913,2024,5,-3.4730,FINNIFTY,40751631.4092,407.5163,408.3892,-0.8728


In [158]:
stats3 = generate_stats(combined_df, "...")
roi_overall, stats_overall = next(iter(stats3.items()))
stats_overall

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2017,30.2769,395,45.0633,0.4744,-0.2496,-4.7040,6.4365,...
2018,34.4350,345,48.6957,0.5755,-0.3517,-12.9533,2.6584,...
2019,35.0491,338,48.2249,0.6392,-0.3951,-8.6832,4.0364,...
2020,116.3527,378,50.7937,1.2078,-0.6212,-11.4869,10.1291,...
2021,56.2923,347,51.8732,0.7430,-0.4638,-10.1526,5.5446,...
2022,66.3831,636,40.4088,0.7973,-0.3655,-12.1918,5.4449,...
2023,57.5092,687,42.0670,0.5395,-0.2472,-8.2939,6.9339,...
2024,27.1147,270,44.4444,0.6681,-0.3537,-6.7273,4.0305,...
Overall,423.4130,3396,45.5536,0.7059,-0.3616,-12.9533,32.6877,...


# MIDFAIR Stats -> BNF, NF, MIDCP, FNF

In [159]:
combined_df.to_csv('MIDFAIR_JJ.csv', index=False)